In [1]:
from func_adl_servicex_xaodr25 import FuncADLQueryPHYSLITE
from servicex import deliver, ServiceXSpec, Sample, dataset
from servicex_analysis_utils import to_awk
from func_adl import ObjectStream, func_adl_callable
from func_adl_servicex_xaodr25.trigger import _add_decision_tool
import ast

from typing import TypeVar, Tuple
T = TypeVar('T')

In [2]:
from typing import List


def _tdt_fired_chains(
    s: ObjectStream[T], a: ast.Call
) -> Tuple[ObjectStream[T], ast.Call]:
    """Configure the backend to run the Trigger Decision Tool to get a list of chain

    Args:
        s (ObjectStream[T]): The stream func_adl is working on - and we can add meta data to.Tuple
        a (ast.Call): The call site in case we need to modify it

    Returns:
        Tuple[ObjectStream[T], ast.Call]: Update stream and call site.
    """
    # The TDT is very slow when it has to fetch a list of all chains in the file. Since
    # this is ServiceX, and we run on only one file at a time, we will cache the list of
    # chains in the file.
    new_s = s.MetaData(
        {
            "metadata_type": "add_cpp_function",
            "name": "tdt_chains_fired",
            "include_files": ["iostream"],
            "arguments": [],
            "code": [
                'static std::vector<std::string> all_triggers;',
                'if (all_triggers.empty()) {',
                '  all_triggers = m_trigDec->getChainGroup(".*")->getListOfTriggers();',
                '}',
                "std::vector<std::string> result;",
                "for (const auto& trigger : all_triggers) {",
                "    if (m_trigDec->isPassed(trigger)) {",
                "        result.push_back(trigger);",
                "    }",
                "}",
            ],
            "result_name": "result",
            "return_type": "std::vector<std::string>",
        }
    )

    return _add_decision_tool(new_s), a


@func_adl_callable(_tdt_fired_chains)
def tdt_chains_fired() -> List[str]:
    """Returns true if the event has any of the trigger chain names that have
    fired. Uses the ATLAS Trigger Decision Tool to query the event.

    Args:
        triggers (str): String specifying the triggers to check for. This is passed directly to the
        ATLAS TriggerDecisionTool, so can include any valid wildcards.

    Returns:
        bool: True if the TDT says this chain has fired on this event, false other wise.
    """
    ...

In [3]:
# Do the local deliver setup
from servicex_local import LocalXAODCodegen, WSL2ScienceImage, SXLocalAdaptor
from servicex_local import deliver as l_deliver

science = WSL2ScienceImage("atlas_al9", "25.2.41")
codegen = LocalXAODCodegen()
adaptor = SXLocalAdaptor(codegen, science, "atlasr25", "http://myserver:8080")

# Comment this next line out to use the official ServiceX service
# deliver = lambda *args, **kwargs: l_deliver(*args, adaptor=adaptor, **kwargs)

# turn on INFO level logging
import logging
# logging.basicConfig(level=logging.INFO)

In [5]:
# Define base query using PHYSLITE
base_query = FuncADLQueryPHYSLITE()

# Simplified query: Select events with at least two jets with pT > 40 GeV
# Then extract jets and MET
query = (
    base_query.Select(lambda e: {
        "triggers": tdt_chains_fired(), 
    })
)

# Dataset name - using a known good PHYSLITE dataset
ds_name = "data24_13p6TeV:data24_13p6TeV.00484979.physics_Late.deriv.DAOD_LLP1.r16740_p6815_p7079_tid47621147_00"
# ds_name = "mc23_13p6TeV:mc23_13p6TeV.562155.MGPy8EG_A14N23LO_ttbar_LL_ALP_fixed_ma40_ctau1500.deriv.DAOD_LLP1.e8599_s4369_r16083_p6876"
ds_file = "../DAOD_LLP1.47621147._000005.pool.root"
ds = dataset.Rucio(ds_name)
# ds = dataset.FileList([ds_file])

# Execute the query
result = deliver(
    ServiceXSpec(
        Sample=[
            Sample(
                Name="triggers",
                Dataset=ds,
                NFiles=1,
                Query=query,
            )
        ]
    ),
)

# Convert to awkward arrays
awk_data = to_awk(result)
data = awk_data["triggers"]

Output()

In [7]:
data['triggers']

<Array [[...], [...], [...], ..., [...], [...]] type='4413 * var * string'>

In [8]:
import pandas as pd
from collections import Counter

# Flatten the nested arrays and count occurrences of each trigger
trigger_counts = Counter()
for event_triggers in data['triggers']:
    trigger_counts.update(event_triggers)

# Create a DataFrame with triggers and their counts
trigger_df = pd.DataFrame([
    {'Trigger': trigger, 'Event Count': count}
    for trigger, count in trigger_counts.items()
]).sort_values('Event Count', ascending=False).reset_index(drop=True)

trigger_df

,Trigger,Event Count
0,HLT_noalg_L1Bkg,4145
1,HLT_larpsall_L1jJ30_FIRSTEMPTY,4083
2,HLT_noalg_LArPEBNoise_L1jJ30_FIRSTEMPTY,4083
3,HLT_noalg_L1jJ60_FIRSTEMPTY,4083
4,HLT_noalg_LArPEBNoise_L1jJ60_FIRSTEMPTY,4083
5,HLT_j55c_xe50_cell_L1jJ60_FIRSTEMPTY,4083
6,L1_jJ60_FIRSTEMPTY,4083
7,L1_jJ30_FIRSTEMPTY,4083
8,HLT_larpsall_L1jJ60_FIRSTEMPTY,4064
9,L1_eEM9_FIRSTEMPTY,1501
